In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Gradient Descent (Advanced)

In this exercise, we will

- Code our Gradient Descent in vectorized form for a high-dimensional Loss Function
- Fine-tune your choice of # of epochs on GD

## 1. Our Dataset

We are going to study the [diabetes dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset) and try to predict the **intensity of the disease** based on **10 quantitative features**, such as body-mass-index, age, etc. (regression problem)

In [ ]:
X, y = datasets.load_diabetes(return_X_y = True, as_frame = True)

print(X.shape)
print(y.shape)

In [ ]:
X.head()

In [ ]:
sns.histplot(y, kde = True);

## 2. Code a Vectorial Gradient Descent

We're modeling a linear regression $\hat{y} = X\beta$

<img src="https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/vectorial-gradient.jpg">

So, first, let's add an "intercept" column of "ones" to our feature matrix X

In [ ]:
# Let's add an intercept column of "ones" 
X = np.hstack((X, np.ones((X.shape[0], 1))))
X.shape

In [ ]:
pd.DataFrame(X).head()

We've created a train/test split for you with `test_size=0.3` and `random_state=1` (so that we all have repeatable results)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)


Let's recall the definition of the gradient descent algorithm

$$\text{Gradient descent - vector formula}$$
$$\beta^{\color {red}{(k+1)}} = \beta^{\color {red}{(k)}} - \eta \ \nabla L(\beta^{\color{red}{(k)}})$$

The MSE Loss for an OLS regression is

$$L(\beta) = \frac{1}{n}\|X \beta - y\|^2 = \frac{1}{n}(X \beta - y)^T(X \beta - y)$$

and its gradient is
$${\displaystyle \nabla L(\beta)=
{\begin{bmatrix}{\frac {\partial L}{\partial \beta_{0}}}(\beta)\\\vdots \\{\frac {\partial L}{\partial \beta_{p}}}(\beta)\end{bmatrix}} = \frac{2}{n} X^T (X\beta - y) 
}$$

Let's store our main problem parameters below:

In [ ]:
# n observations
n = X.shape[0] 
n_train = X_train.shape[0]
n_test = X_test.shape[0]

# p features (including the intercept)
p = X.shape[1]

# Gradient Descent hyper-params
eta = .1
n_epochs= 100

❓ Initialize a $\beta$ vector of zeros of shape **p**

In [ ]:
# YOUR CODE HERE

❓ Using the vectorized formula given above, create a Gradient Descent that loops over `n_epochs` to find the best $\beta$ of an OLS using the `train` set
- make use of NumPy's matrix operations and broadcasting capabilities
- this shouldn't take more than 4 lines of code!

In [ ]:
for epoch in range(n_epochs):
    gradient = 2 / n_train * np.dot(X_train.T, (np.dot(X_train, beta) - y_train))
    beta = beta - eta * gradient

In [ ]:
print('Best ß: ', beta)

## Predict

❓Compute predictions on your test set (`y_pred`), and the resulting `loss_test` (MSE loss for OLS).

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

## Wrap these into a function called `gradient_decent`

❓ Wrap this logic into a function called `gradient_descent`, which takes as input some (`X_train`, `y_train`, `X_test`, `y_test`, `eta`, `n_epoch`) values, and returns:
- the final value for $\beta$ fitted on the train set
- the values of the `loss_train` at each epoch as a list called `loss_train_history`
- the values of the `loss_test` at each epoch as a list called `loss_test_history`
- (optional) make the function robust to call with only a train_set

In [ ]:
def gradient_descent(X_train, y_train, X_test, y_test, eta = eta, n_epochs = 100):
    n_train = X_train.shape[0]
    n_test = X_test.shape[0]
    p = X_train.shape[1]

    beta = np.zeros(p)

    loss_train_history = []
    loss_test_history = []

    pass  # YOUR CODE HERE

    return beta, loss_train_history, loss_test_history

## Early stopping criteria?

❓Plot the loss as a function of epochs, on your train dataset. 
- Try it with `n_epochs=10000` and `eta=0.1` as was initially set
- Zoom in with `plt.ylim(ymin=2800, ymax=3000)` to see the behavior of the Loss Function on the test set
- What can you conclude? Should you always "descend" the gradient down to the absolute minimum?

In [ ]:
# YOUR CODE HERE

In [ ]:
# Plot train and test histories
plt.plot(loss_train_history, label = 'loss_train')
plt.plot(loss_test_history, label = 'loss_test')

# Set title and labels
plt.title('Loss')
plt.ylabel('MSE Loss')
plt.xlabel('Epochs')

# Change limits
plt.ylim(ymin = 2800, ymax = 3000)

# Generate legend
plt.legend()

❓ What do you notice?

> YOUR ANSWER HERE

❓Can you think of a method to improve the performance of your model? Take time to write it in pseudo-code below before looking at the hints.

<details>
    <summary>Hints</summary>

- We could decide to stop the GD as soon as the test loss starts to increase again.
- ⚠️ Yet we can't use the "test set" created initially to decide when to stop descending gradient; this would create data leakage! Never use your test set to optimize your model's `hyperparameters`.
- Create instead a train/test split **within** your current training set and optimize your early stopping based on the loss of this new test set only. This one is usually called a **validation set**. 
</details>

In [ ]:
#PSEUDO-CODE

❓ Update your `gradient_descent` method based on the Hints above!

In [ ]:
# YOUR CODE HERE

❓ Create your train/val set and try to improve your MSE with early stopping, using `random_state = 1`

It should stop earlier than before!

In [ ]:
# YOUR CODE HERE

In [ ]:
beta_es, loss_train_history, loss_val_history = gradient_descent_early_stopping(X_train_train, y_train_train, X_val, y_val, n_epochs = 10000, eta = .1)

# Plot train and test histories
plt.plot(loss_train_history, label = 'loss_train')
plt.plot(loss_test_history, label = 'loss_test')

# Set title and labels
plt.title('Loss')
plt.ylabel('MSE Loss')
plt.xlabel('Epochs')

# Change limits
plt.ylim(ymin = 2500, ymax = 4000)

# Generate legend
plt.legend()

## Mini-Batch Descent

❓ Modify your gradient_descent function into a `minibatch_gradient_descent` one.

In [ ]:
def minibatch_gradient_descent(X_train, y_train, X_test, y_test, batch_size = 16, eta = eta, n_epochs = n_epochs):
    n_train = X_train.shape[0]
    n_test = X_test.shape[0]

    p = X_train.shape[1]

    beta = np.zeros(p)

    loss_train_history = []
    loss_test_history = []

    if isinstance(y_train, pd.Series):
        y_train = y_train.to_numpy()

    for epoch in range(n_epochs):
        # Shuffle your (X_train, y_train) dataset
        pass  # YOUR CODE HERE

        # Loop over your dataset in mini-batches, and for each mini-batch update your beta
        pass  # YOUR CODE HERE

        # Keep track of loss histories per epoch
        pass  # YOUR CODE HERE

    return beta, loss_train_history, loss_test_history

❓ Plot the evolution of your train and val losses per epoch. What if you chose minibatch = 1?

In [ ]:
# YOUR CODE HERE

❓ How would you adjust the early stopping criteria to these fluctuations?

<details>
    <summary>Hint</summary>

To avoid early stopping too early due to the stochastic nature of the mini-batch descent, we could add a "patience" term to stop only after the val loss is increased for a sustained period of "patience" # of epochs.
</details>

## Conclusion: a new way to check for overfitting

<img src="https://wagon-public-datasets.s3-eu-west-1.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/underfitting_overfitting_at_a_glance.webp" width=800>

📚 Read more about:

- [Underfitting and overfitting](https://towardsdatascience.com/overfitting-vs-underfitting-a-complete-example-d05dd7e19765)
- [Gradient Descent in Python](https://towardsdatascience.com/gradient-descent-in-python-a0d07285742f)

# 🏁 Congrats on completing this challenge! Time to push your notebook